# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante 

### SAPORI SOLARI - Milano

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187849-d8780418-Reviews-Sapori_Solari-Milan_Lombardy.html

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187849-d8780418-Reviews-Sapori_Solari-Milan_Lombardy.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Executed")

Executed


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, commenti e della prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
### SUPERFLUO

### SECONDO SCRAPING (pagina 8)

Capiamo come fare!

In [16]:
URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187849-d8780418-Reviews-or70-Sapori_Solari-Milan_Lombardy.html'

all_ratings = []
all_reviews = []
all_names = []

# OTTENGO I DATI E CI APPLICO BEATIFULSOUP
data_html = requests.get(URL_COMPLETO)
data_html_soup = BeautifulSoup(data_html.content, "lxml")
    

############################################################################################################################
### REVIEWS
        
# all_reviews_html = data_html_soup.select('div.review-container p.partial_entry')
        
# all_reviews_html = data_html_soup.findAll("div", {"class": "entry"})     # Test usando BeautifulSoup
    
all_reviews_and_answers_html = data_html_soup.findAll("div", attrs={"class":"wrap"})

for element in all_reviews_and_answers_html:
    risposta = element.find('p', attrs={'class': 'partial_entry'})
    if risposta:
        all_reviews.append(risposta)
     
############################################################################################################################
### NAMES
        
all_names_html = data_html_soup.select("div.review-container span.expand_inline.scrname");    
for name in all_names_html:
    all_names.append(name.text)
    
    
############################################################################################################################
### RATINGS
    
# all_ratings_html = data_html_soup.select("div.review-container span.ui_bubble_rating")

all_ratings_html = data_html_soup.findAll("div", attrs={"class":"review-container"})

for element in all_ratings_html:
    voto = element.find('span', attrs={'class': 'ui_bubble_rating'})
    if voto:
        all_ratings.append(voto['class'][1][7:8])
        
################################################################################################################################
        
print("Executed\n")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))

Executed

Nomi:  10
Commenti:  10
Voti:  10


In [11]:
#display(all_reviews)

In [6]:
print(all_names)

['Sofia S', 'federica_m00', 'Skdufyt', 'cicabum', 'elecar76', 'marcogK4706US', 'ladyoscar007', 'Andrea T', 'virgiliodemidio', 'vanymi']


In [17]:
print(all_ratings)

['5', '5', '4', '5', '5', '5', '4', '4', '2', '5']


#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [8]:
filtered_reviews = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        filtered_reviews.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        filtered_reviews.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate)

display(filtered_reviews)

Totale recensioni: 10 
 Recensioni analizzabili: 10 
 Recensioni scartate: 0


[<p class="partial_entry">Salumi e formaggi particolari e deliziosi. Servizio impeccabile e personale amichevole.
 Da provare!!</p>,
 <p class="partial_entry">Siamo clienti abituali di questo bellissimo e buonissimo locale. Non delude mai: non deludono i sapori (cibo e vino, sempre una sicurezza!); non deludono i ragazzi - gentilezza che a Milano è rara; sorrisi sinceri e entusiasmo tangibile. A Milano non c'è niente di così...<span class="taLnk ulBlueLinks" onclick="widgetEvCall('handlers.clickExpand',event,this);">Più</span></p>,
 <p class="partial_entry">salumi ricercati da piu' parti di italia, camerieri preparati e gentili. carta vini migliorata ma ancora migliorabile.</p>,
 <p class="partial_entry">Ci siamo trovati benissimo. È una formula interessante, affettati di carne e pesce eccellenti e atmosfera piacevolissima. Ottimo arredo, ottima ospitalità e dolci buonissimi fatti in casa! Si capisce che c’è amore e ricerca.</p>,
 <p class="partial_entry">Ricercato ma accogliente. Raff

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [9]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)          # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['Sofia S', 'federica_m00', 'Skdufyt', 'cicabum', 'elecar76', 'marcogK4706US', 'ladyoscar007', 'Andrea T', 'virgiliodemidio', 'vanymi'] 

['Sofia', 'federicam', 'Skdufyt', 'cicabum', 'elecar', 'marcogKUS', 'ladyoscar', 'Andrea', 'virgiliodemidio', 'vanymi']



#### Utilizzo Genderize
Infine, posso utilizzare la libreria Genderize.

In [10]:
from genderize import Genderize

names_with_gender = []

for name in all_names_filtered:
    gender = Genderize().get([name])
    names_with_gender.append(gender)
  
display(names_with_gender)
# Ottengo una lista [ [lista con dizionario {} all'interno] ]

GenderizeException: ('Request limit reached', 429, {'Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Thu, 27 Dec 2018 18:49:36 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '33', 'Connection': 'keep-alive', 'X-Powered-By': 'Express', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET', 'Access-Control-Allow-Headers': 'Content-Type', 'ETag': 'W/"21-/l1SSO33EFsZ7fO3uQLHxw"'})

In [ ]:
all_genders = []
i = 0

for element in names_with_gender:
    # print(names_with_gender[i][0]) --> {'name': 'Sabrina', 'gender': 'female', 'probability': 1.0, 'count': 1331}
    
#   if names_with_gender[i][0]['gender'] == None or names_with_gender[i][0]['probability'] < 0.7:
#   CONTROLLO QUALITA' CHE NON POSSIAMO PERMETTERCI A QUANTO PARE

    if names_with_gender[i][0]['gender'] == None:
        all_genders.append('UNKNOWN')
    elif names_with_gender[i][0]['gender'] == 'male':
        all_genders.append('M')
    else:
        all_genders.append('F')
    
    # passo all'elemento successivo
    i+=1
    
display(all_genders)

---
### Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



In [ ]:
# import pandas as pd

# tabella = pd.read_excel("PARTE_4_Save_the_data.xlsx")

# display(tabella)

### Controlliamo che ci siano lo stesso numero di nomi, commenti e voti
Potremo così creare una tabella sfruttando Pandas

In [ ]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))

In [ ]:
import pandas as pd

# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nome': all_names_filtered, 'Genere': all_genders})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('TEST_1_NOMI_E_GENERE.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Review': filtered_reviews, 'Voti': all_ratings})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('TEST_1_REVIEW_E_VOTI.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))

# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nome': all_names[:235], 'Review': all_reviews[:235], 'Voto': all_ratings[:235]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('TEST_1_TUTTI_I_DATI.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()